# Water Quality CSV Transects joined by TIME/DATE 

In [1]:
import os
import pandas
from scripts import wqt_timestamp_match
from datetime import datetime

In [2]:
# set wd to Arcproject-wq-processing folder
wd = os.path.abspath(os.path.join(os.path.dirname(os.path.dirname("__file__"))))
# example shp
shp = os.path.join(wd, "scripts", "tests", "testfiles", "Arc_040413\\Arc_040413_GPS\\040413_PosnPnt.shp")

# example water quality
wq_file = os.path.join(wd, "scripts", "tests", "testfiles", "Arc_040413\\Arc_040413_WQ\\Arc_040413_wqt_cc.csv")


print(shp)
print(wq_file)

C:\Users\Andy\Documents\arcproject-wq-processing\scripts\tests\testfiles\Arc_040413\Arc_040413_GPS\040413_PosnPnt.shp
C:\Users\Andy\Documents\arcproject-wq-processing\scripts\tests\testfiles\Arc_040413\Arc_040413_WQ\Arc_040413_wqt_cc.csv


### Import CSV file from the Sonde as Pandas Dataframe

In [3]:
water_quality = wqt_timestamp_match.wq_from_file(wq_file)
print(water_quality.head())

            Date_Time   Temp    pH SpCond   Sal DEP25  PAR RPAR TurbSC   CHL  \
1 2013-04-04 13:01:03  17.40  7.84    520  0.26  0.39  573  417   13.4  5.34   
2 2013-04-04 13:01:04  17.40  7.84    520  0.26  0.35  573  417   13.4  5.34   
3 2013-04-04 13:01:05  17.40  7.88    520  0.26  0.35  576  419   13.5  5.29   
4 2013-04-04 13:01:06  17.41  7.85    521  0.26  0.39  576  417   13.5  5.31   
5 2013-04-04 13:01:07  17.41  7.85    521  0.26  0.39  576  417   13.5  5.31   

  CHL_VOLTS              WQ_SOURCE  
1   0.05078  Arc_040413_wqt_cc.csv  
2   0.05078  Arc_040413_wqt_cc.csv  
3   0.05095  Arc_040413_wqt_cc.csv  
4   0.05073  Arc_040413_wqt_cc.csv  
5   0.05073  Arc_040413_wqt_cc.csv  


### Transect Shapefile 

In [4]:
shp_df = wqt_timestamp_match.wqtshp2pd(shp)

print(shp_df)

print(shp_df.dtypes)

         GPS_Date    GPS_Time     POINT_X    POINT_Y          GPS_SOURCE  \
0      2013-04-04  08:18:47am -121.675982  38.233225  040413_PosnPnt.shp   
1      2013-04-04  08:18:48am -121.675986  38.233220  040413_PosnPnt.shp   
2      2013-04-04  08:18:49am -121.675990  38.233214  040413_PosnPnt.shp   
3      2013-04-04  08:18:50am -121.675994  38.233209  040413_PosnPnt.shp   
4      2013-04-04  08:18:51am -121.675997  38.233204  040413_PosnPnt.shp   
5      2013-04-04  08:18:52am -121.676001  38.233199  040413_PosnPnt.shp   
6      2013-04-04  08:18:53am -121.676004  38.233193  040413_PosnPnt.shp   
7      2013-04-04  08:18:54am -121.676006  38.233188  040413_PosnPnt.shp   
8      2013-04-04  08:18:55am -121.676009  38.233183  040413_PosnPnt.shp   
9      2013-04-04  08:18:56am -121.676011  38.233177  040413_PosnPnt.shp   
10     2013-04-04  08:18:57am -121.676013  38.233172  040413_PosnPnt.shp   
11     2013-04-04  08:18:58am -121.676015  38.233166  040413_PosnPnt.shp   
12     2013-

### Join the transect geodataframe with the water quality dataframe

In [5]:
merged = wqt_timestamp_match.JoinByTimeStamp(water_quality, shp_df)
print(merged.head())

print(shp_df.shape)
print(water_quality.shape)
print(merged.shape)


#match = wqt_timestamp_match.splitunmatched(merged)
joined_data = merged
match = joined_data.dropna(axis='index')
no_geo = joined_data[joined_data["GPS_SOURCE"].isnull()]
no_wq = joined_data[joined_data["WQ_SOURCE"].isnull()]

print(match.shape)
print(no_geo.shape)
print(no_wq.shape)

     GPS_Date    GPS_Time     POINT_X    POINT_Y          GPS_SOURCE  \
0  2013-04-04  08:18:47am -121.675982  38.233225  040413_PosnPnt.shp   
1  2013-04-04  08:18:48am -121.675986  38.233220  040413_PosnPnt.shp   
2  2013-04-04  08:18:49am -121.675990  38.233214  040413_PosnPnt.shp   
3  2013-04-04  08:18:50am -121.675994  38.233209  040413_PosnPnt.shp   
4  2013-04-04  08:18:51am -121.675997  38.233204  040413_PosnPnt.shp   

            Date_Time Temp   pH SpCond  Sal DEP25  PAR RPAR TurbSC  CHL  \
0 2013-04-04 08:18:47  NaN  NaN    NaN  NaN   NaN  NaN  NaN    NaN  NaN   
1 2013-04-04 08:18:48  NaN  NaN    NaN  NaN   NaN  NaN  NaN    NaN  NaN   
2 2013-04-04 08:18:49  NaN  NaN    NaN  NaN   NaN  NaN  NaN    NaN  NaN   
3 2013-04-04 08:18:50  NaN  NaN    NaN  NaN   NaN  NaN  NaN    NaN  NaN   
4 2013-04-04 08:18:51  NaN  NaN    NaN  NaN   NaN  NaN  NaN    NaN  NaN   

  CHL_VOLTS WQ_SOURCE  
0       NaN       NaN  
1       NaN       NaN  
2       NaN       NaN  
3       NaN       Na

In [7]:
import arcpy
import numpy

outFC = r"C:\Users\Andy\Desktop\testShape.shp"  
array = numpy.array([(1, 'testText1', (471316.3835861763, 5000448.782036674)),  
                     (2, 'testText2', (470402.49348005146, 5000049.216449278))],  
                    numpy.dtype([('idfield',numpy.int32),('textField','<U10'),('XY', '<f8', 2)]))  
 
print(array)    
    
arcpy.da.NumPyArrayToFeatureClass(array, outFC, ['XY'])

[(1, u'testText1', [471316.3835861763, 5000448.782036674])
 (2, u'testText2', [470402.49348005146, 5000049.216449278])]


In [ ]:
# save to csv format?
csv = match.to_csv()

# save to numpy format? and then convert to feature class in arcpy 
# see https://my.usgs.gov/confluence/display/cdi/pandas.DataFrame+to+ArcGIS+Table



df = match


import numpy as np

x = np.array(np.rec.fromrecords(df.values))
names = df.dtypes.index.tolist()
x.dtype.names = tuple(names)


x = x.astype([('GPS_Date', 'S10'), ('GPS_Time', 'S10'), ('POINT_X', '<f8'), ('POINT_Y', '<f8'), ('GPS_SOURCE', 'S18'),
              ('Date_Time', '<M8[us]'), ('Temp', '<f'), ('pH', '<f'), ('SpCond', '<f'), ('Sal', '<f'), ('DEP25', '<f'), 
              ('PAR', '<f'), ('RPAR', '<f'), ('TurbSC', '<f'), ('CHL', '<f'), ('CHL_VOLTS', '<f'), ('WQ_SOURCE', 'S21')])


print(x.dtype)

In [ ]:
field_type_dict = {'GPS_Time': 'S10', 'Date_Time': '<M8[us]', 'PAR': '<f', 'Temp': '<f', 'Sal': '<f', 'DEP25': '<f', 'CHL': '<f', 'TurbSC': '<f', 'RPAR': '<f', 'SpCond': '<f', 'POINT_X': '<f8', 'POINT_Y': '<f8', 'GPS_Date': 'S10', 'pH': '<f', 'WQ_SOURCE': 'S21', 'GPS_SOURCE': 'S18', 'CHL_VOLTS': '<f'}

print(field_type_dict)

print(field_type_dict.items())

In [ ]:
import arcpy

out = r"C:\Users\Andy\Desktop\ex.shp" 
arcpy.da.NumPyArrayToFeatureClass(r, out, ("POINT_X", "POINT_Y"))

In [10]:
a = wqt_timestamp_match.pd2np(match, wqt_timestamp_match.field_type_dict.items())
wqt_timestamp_match.np2feature(a, r"C:\Users\Andy\Desktop\b.shp" )